#  <center> Problem Set 6 <center>
<center> Spring 2022 <center>
<center> 3.C01, 10.C01, 20.C01 <center>
<center> Due: 5 PM ET on Monday, May 9, 2022 <center>

In [ ]:
# This is a hack to install rdkit without needing to install conda which might take minutes 
# if you have anaconda installed, you can install rdkit from anaconda 
url = 'https://anaconda.org/rdkit/rdkit/2018.09.1.0/download/linux-64/rdkit-2018.09.1.0-py36h71b666b_1.tar.bz2'
!curl -L $url | tar xj lib
!mv lib/python3.6/site-packages/rdkit /usr/local/lib/python3.7/dist-packages/

x86 = '/usr/lib/x86_64-linux-gnu'
!mv lib/*.so.* $x86/
!ln -s $x86/libboost_python3-py36.so.1.65.1 $x86/libboost_python3.so.1.65.1

Register for the competition and download the data: 

https://www.kaggle.com/t/5ebc2904c2de406a8309cd86eb34015b

In [ ]:
!wget https://raw.githubusercontent.com/vikram-sundar/ML4MolEng_Spring2022/master/psets/ps6-molecule/data/molecule_props.csv
!wget https://raw.githubusercontent.com/vikram-sundar/ML4MolEng_Spring2022/master/psets/ps6-molecule/data/solvation_train.csv
!wget https://raw.githubusercontent.com/vikram-sundar/ML4MolEng_Spring2022/master/psets/ps6-molecule/data/solvation_test.csv

--2021-05-05 23:35:40--  https://raw.githubusercontent.com/wwang2/ML4MolEng/master/psets/ps6/data/chem_mat/molecule_props.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26739 (26K) [text/plain]
Saving to: ‘molecule_props.csv’

molecule_props.csv  100%[===================>]  26.11K  --.-KB/s    in 0s      

2021-05-05 23:35:40 (52.6 MB/s) - ‘molecule_props.csv’ saved [26739/26739]

--2021-05-05 23:35:40--  https://raw.githubusercontent.com/wwang2/ML4MolEng/master/psets/ps6/data/chem_mat/solvation_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting

## Part 1: Baseline Regression Methods

### Part 1.1: (10 points) Prepare Dataset

In [ ]:
import pandas as pd 

# load data 
data = pd.read_csv('./solvation_train.csv')
mol_prop = pd.read_csv('./molecule_props.csv')

Some utility functions for you to generate features.

In [ ]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors,Crippen
import numpy as np

Generate fingerprints (e.g. a Morgan fingerprint).

In [ ]:
# Get molecular feature from RDkit 
smiles = 'CN1C=NC2=C1C(=O)N(C(=O)N2C)C'

# define Mol object
mol = Chem.MolFromSmiles(smiles)

# get morgan fingerprint 
# obtain a 512 bit fingperint, with radius 2
fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=512) 

# convert to numpy array 
fp_array = np.zeros((1,), int)
DataStructs.ConvertToNumpyArray(fp, fp_array)

(512,)


Generate various chemical properties.

In [ ]:
MolWt = Descriptors.ExactMolWt(mol)
TPSA = Chem.rdMolDescriptors.CalcTPSA(mol) #Topological Polar Surface Area
nRotB = Descriptors.NumRotatableBonds(mol) #Number of rotable bonds
HBD = Descriptors.NumHDonors(mol) #Number of H bond donors
HBA = Descriptors.NumHAcceptors(mol) #Number of H bond acceptors
logP = Descriptors.MolLogP(mol) #LogP

Create a feature set with concatenated physical descriptors.

### 1.2 (10 points) Linear Regression

Train a linear regression model and report a 5-fold cross-validated R^2.

### 1.3 (10 points) MLP Regression

Train an MLP regression model and report a 5-fold cross-validated R^2.

## Part 2: (70 points) Machine Learning Competition and Report

You can start a new notebook here to put all your models.

In [ ]:
def save_submission(prediction, filename):
    '''
    Utility function to dump a submission file.
    
    prediction (numpy.array): 1d numpy array contains your prediction
    filename (str): file path to where you want to save the result
    '''
    sub = pd.DataFrame( {'index': list(range(len(prediction))), 'logK': prediction } )
    sub.to_csv(filename, index=False)